<a href="https://colab.research.google.com/github/Ramyan-Iz/KAA/blob/main/TRUESHIIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl scikit-learn  # openpyxl для чтения Excel


In [ ]:
# Загрузка Excel-файла
df = pd.read_excel('VIBORKA.xlsx', header=None, names=['raw_data'])  # Читаем как одну колонку
print(df.head())
# Разделяем по последней запятой
df[['text', 'label']] = df['raw_data'].str.rsplit(',', n=1, expand=True)

# Удаляем лишние пробелы
df['text'] = df['text'].str.strip()
df['label'] = df['label'].str.strip().astype(float)  # Преобразуем в число

# Проверяем результат
print(df.head())
print(f"Всего строк: {len(df)}")
print(f"Распределение классов:\n{df['label'].value_counts()}")
print(f"Есть ли пропуски:\n{df.isnull().sum()}")


                                            raw_data
0  If I cannot even find one situation, then I am...
1              I cannot believe I ate that cake.,2.0
2  I found Jason s observation very helpful in re...
3  I have found myself feeling hopeless about the...
4  I feel distraught and completely tormented eve...
                                            raw_data  \
0  If I cannot even find one situation, then I am...   
1              I cannot believe I ate that cake.,2.0   
2  I found Jason s observation very helpful in re...   
3  I have found myself feeling hopeless about the...   
4  I feel distraught and completely tormented eve...   

                                                text  label  
0  If I cannot even find one situation, then I am...    7.0  
1                  I cannot believe I ate that cake.    2.0  
2  I found Jason s observation very helpful in re...    2.0  
3  I have found myself feeling hopeless about the...    7.0  
4  I feel distraught and completely

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Разделяем данные на train и test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Инициализируем токенизатор DeBERTa v3
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Преобразуем DataFrame в Dataset (удобный формат для HuggingFace)
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/24902 [00:00<?, ? examples/s]

Map:   0%|          | 0/6226 [00:00<?, ? examples/s]

In [ ]:
import transformers
print(transformers.__version__)


4.51.3


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Загружаем модель
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=8  # У нас 8 классов (0-7)
)

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# Аргументы обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

# Обучаем модель
trainer.train()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rds7 (rds7-tpu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
